In [6]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [7]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [8]:
pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [9]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [10]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [11]:
#Подключение модулей
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [12]:
#Процедура для подготовки данных      
def vectorize(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [13]:
#Процедура для предобработки данных
def preprocess(dimension):
    #Загрузим датесет IMDb, который уже встроен в Keras
    (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=dimension)
    data = np.concatenate((X_train, X_test), axis=0)
    targets = np.concatenate((y_train, y_test), axis=0)
    #Вызываем процедуру для подготовки данных
    data = vectorize(data, dimension)
    #Преобразование переменных в тип float32
    targets = np.array(targets).astype("float32")
    #Возвращаем разделённый датасет на обучающий и тестировочный наборы
    return data[dimension:], targets[dimension:], data[:dimension], targets[:dimension]

In [14]:
#Процедура для предобработки данных для тестирования модели     
def encode(words):
    #при помощи метода get_word_index() получаем числовые индексы для слов, известных датасету imdb
    word_index = imdb.get_word_index()
    my_word_index = {}
    for k, v in word_index.items():
        my_word_index[k]= v+3
    arr = []
    for word in words:
        arr.append(my_word_index.get(word, 2))
    #Максимальная длина отзыва 17    
    res = pad_sequences([arr], value=0, padding="post",maxlen=17)
    return res

In [15]:
#Процедура для разбиения предложение на слова, а затем выполняет преобразование созданной функцией encode
def text_pred(text, dimension, folder):
    words = text.split()
    result = encode(words)
    vector_text = vectorize(result, dimension)
    model = load_model(folder)
    print("")
    print("Текст - ", text)
    print("Вероятность положительности отзыва: " + str(model.predict(vector_text)[0][0]))

In [16]:
#Процедура для вывода результатов
def print_results(dimension, accuracy, loss):
    print("")
    print("Размер вектора представления - " + str(dimension))
    print('Точность на тестовых данных:', accuracy)
    print('Потери на тестовых данных:', loss)

In [17]:
#Процедура для построения и настройки параметров компиляции была разработана функция
def create_model(X_train, y_train, X_test, y_test, dimension, folder):
    #ИНС состояла из двух повторяющихся блоков слоев (2 раза подряд полносвязный слой из 50 нейронов и разреживания (0.3 и 0.2))
    model = Sequential()
    model.add(Dense(50, activation = "relu", input_shape=(dimension, )))
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2, noise_shape=None, seed=None))
    #Для решения задачи бинарной классификации использовался выходной слой из 1 нейрона с функцией активации
    model.add(Dense(1, activation = "sigmoid"))
    #В качестве оптимизатора использовалась «Adam», функции потерь «бинарная перекрестная энтропия», метрики – «accuracy». Модель сохранялась в каталоге с названием folder. 
    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    history = model.fit(X_train, y_train, epochs = 2, batch_size = 500, validation_data = (X_test, y_test))
    model.save(folder)
    #Данная функция возвращает последние значения точности и потерь на тестовой выборке из истории обучения
    return history.history['val_accuracy'][-1], history.history['val_loss'][-1]

In [21]:
#Исследуем результаты при различном размере вектора представления текста и создаём модели для дальнейшего тестирования
X_train1, y_train1, X_test1, y_test1 = preprocess(5000)
accuracy1, loss1 = create_model(X_train1, y_train1, X_test1, y_test1, 5000, 'model1')
print_results(5000, accuracy1, loss1)
X_train2, y_train2, X_test2, y_test2 = preprocess(10000)
accuracy2, loss2 = create_model(X_train2, y_train2, X_test2, y_test2, 10000, 'model2')
print_results(10000, accuracy2, loss2)
X_train3, y_train3, X_test3, y_test3 = preprocess(15000)
accuracy3, loss3 = create_model(X_train3, y_train3, X_test3, y_test3, 15000, 'model3')
print_results(15000, accuracy3, loss3)
X_train4, y_train4, X_test4, y_test4 = preprocess(20000)
accuracy4, loss4 = create_model(X_train4, y_train4, X_test4, y_test4, 20000, 'model4')
print_results(20000, accuracy4, loss4)

Epoch 1/2
90/90 [==============================] - 1s 7ms/step - loss: 0.4070 - accuracy: 0.8170 - val_loss: 0.2691 - val_accuracy: 0.8920
Epoch 2/2
90/90 [==============================] - 1s 6ms/step - loss: 0.2441 - accuracy: 0.9041 - val_loss: 0.2575 - val_accuracy: 0.8960
INFO:tensorflow:Assets written to: model1\assets

Размер вектора представления - 5000
Точность на тестовых данных: 0.8960000276565552
Потери на тестовых данных: 0.2575203776359558
Epoch 1/2
80/80 [==============================] - 1s 13ms/step - loss: 0.4229 - accuracy: 0.8125 - val_loss: 0.2710 - val_accuracy: 0.8928
Epoch 2/2
80/80 [==============================] - 1s 11ms/step - loss: 0.2259 - accuracy: 0.9134 - val_loss: 0.2659 - val_accuracy: 0.8946
INFO:tensorflow:Assets written to: model2\assets

Размер вектора представления - 10000
Точность на тестовых данных: 0.894599974155426
Потери на тестовых данных: 0.26585450768470764
Epoch 1/2
70/70 [==============================] - 1s 19ms/step - loss: 0.4002 - 

In [19]:
#Прогноз успеха фильмов по обзорам
text1 = "this movie isn't that bad"
text_pred(text1, 10000, 'model2')
text2 = "great movie, great acting, amazing scenery"
text_pred(text2, 10000, 'model2')
text3 = "the film is very dark, not interesting to watch, the acting is just terrible"
text_pred(text3, 10000, 'model2')


Текст -  this movie isn't that bad
1/1 [==============================] - 0s 64ms/step
Вероятность положительности отзыва: 0.26349857

Текст -  great movie, great acting, amazing scenery
1/1 [==============================] - 0s 24ms/step
Вероятность положительности отзыва: 0.8065244

Текст -  the film is very dark, not interesting to watch, the acting is just terrible
1/1 [==============================] - 0s 25ms/step
Вероятность положительности отзыва: 0.15477958
